<p align="center">
    <img src="NFL_Banner.jpg" width=100%>
</p>

# NFL-Dataset

### NFL Scorigamis – Einzigartige Endstände im American Football
Ein Scorigami in der NFL bezeichnet ein Endergebnis, das in der Geschichte der Liga noch nie vorgekommen ist. Da es in American Football aufgrund der speziellen Punktevergabe (Touchdowns = 6 Punkte, Field Goals = 3 Punkte, Safeties = 2 Punkte, Extrapunkte & Two-Point Conversions) viele mögliche Endstände gibt, entstehen gelegentlich neue, zuvor unerreichte Kombinationen.

Die Spielregeln beeinflussen Scorigamis erheblich, da die unterschiedlichen Punktwerte und mögliche Kombinationen aus Touchdowns, Field Goals und Safeties zu teilweise ungewöhnlichen Ergebnissen führen können. Besonders seltene Ereignisse wie Safeties oder verfehlte Extrapunkte tragen oft dazu bei, neue Scorigamis zu ermöglichen.

Das Konzept wurde von Jon Bois populär gemacht, und es gibt mittlerweile eine Community, die jedes Spiel verfolgt, um zu sehen, ob ein neues Scorigami erzielt wurde.

### Schritt 1: Benötigte Libraries importieren

In [1]:
# Benötigte Libraries

# Webscraping
import csv
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
from time import sleep
from tqdm import tqdm

# Visualisieren
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

### Schritt 2: Web-Scraping
Es wird nun der Datensatz benötigt, der alle Spiele der NFL jemals beinhaltet. Dazu bietet sich die Website [pro-football-reference.com](https://www.pro-football-reference.com/boxscores/game-scores.htm) an. Sie beinhaltet zunächst alle Endstände sowie ihre Häufigkeit. Mittels `requests` wird die .html-Datei der Seite zunächst heruntergeladen und im Verzeichnis `data/` als `scores.html` gespeichert.

In [1]:
# Datei herunterladen

# URL, die gescraped werden soll
url = "https://www.pro-football-reference.com/boxscores/game-scores.htm"

# Erstelle GET-Request der URL
response = requests.get(url)

# Prüfe, ob Request erfolgreich war
if response.status_code == 200:
    # Speichere Seite in einer HTML-Datei
    with open("data/scores.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Page saved as 'scores.html'")
else:
    print(f"Failed to download page. Status code: {response.status_code}")

Page saved as 'scores.html'


Aus `scores.html` werden nun alle einzelnen Endstände mithilfe von `BeautifulSoup` geparsed und in der Variable `score_list` gespeichert. Basierend auf dieser Liste kann nun auf der o. g. Website auf die Spiele mit jeweiligem Endergebnis zugegriffen werden. Diese Subseiten werden mit dem Namen des Endergebnisses als .html-Datei im Verzeichnis `data/games/` gespeichert. Insgesamt entstehen dabei 1091 .html-Dateien.

In [74]:
# Datei nach Scores Parsen

# If the HTML is saved locally, you can use:
with open("data/scores.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# Initialize an empty dictionary to store the country data
score_list = []

# Find all div elements with class "col-md-4 country"
for scores in soup.select("table.stats_table tbody tr:not([thead])"):
    score = scores.find("td").text
    score_list.append(score)

# Alle Spiele zu jedem Score finden

for score in tqdm(score_list,desc="Scores herunterladen"):
    i, j = score.split("-")
    url = f"https://www.pro-football-reference.com/boxscores/game_scores_find.cgi?pts_win={i}&pts_lose={j}"

    # Erstelle GET-Request der URL
    response = requests.get(url)

    # Prüfe, ob Request erfolgreich war
    if response.status_code == 200:
        # Speichere Seite in einer HTML-Datei
        with open(f"data/games/{i}-{j}.html", "w", encoding="utf-8") as file:
            file.write(response.text)
    else:
        print(f"Failed to download {i}-{j}.html. Status code: {response.status_code}")

    sleep(.75)

Scores herunterladen: 100%|██████████| 1091/1091 [53:59<00:00,  2.97s/it] 


Jetzt, wo die 1091 .html-Dateien lokal gespeichert sind, können auch sie mit BeautifulSoup geparsed werden. Dabei werden zunächst alle columns übernommen und später bereinigt. Die geparseden Ergebnisse werden in der Variable `games_list` gespeichert, die daraufhin in die Datei `data/games.csv` geschrieben werden. Diese Datei enthält nun alle Spiele der NFL samt Ergebnissen, Team-Namen, Spieltagen und vielen weiteren Statistiken (unbereinigt).

In [ ]:
# Alle Spiele
games_list = []

# Alle Spiele in Dateien speichern
for score in tqdm(score_list,desc="Scores kombinieren"):
    i, j = score.split("-")

    # Datei nach jeweiligem Score speichern
    with open(f"data/games/{i}-{j}.html", "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    # Alle Spiele des jeweiligen Scores
    for games in soup.select("table.stats_table tbody tr:not([thead])"):
        game = games.find_all("td")
        games_list.append([i.text.strip() for i in game])

# In CSV games.csv speichern
with open("data/games.csv", "a", newline="") as file:
    writer = csv.writer(file)
    
    fields = ["week", "day", "date", "outcome", "winner", "game_location", "loser", "boxscore", "pts_w", "pts_l", "yards_w", "to_w", "yards_l", "to_l"]
    writer.writerow(fields)
    writer.writerows(games_list)

print("Done!")

Scores kombinieren: 100%|██████████| 1091/1091 [04:20<00:00,  4.19it/s]

Done!


### Schritt 3: Datenbereinigung
Schaut man sich die `games.csv` an, fällt neben anderen Unreinheiten, die gesäubert werden sollten, auf, dass oft Team-Variationen als Namen vorkommen. So spielen bspw. 1944 die *Cleveland Rams*, 1962 aber die *Los Angeles Rams*. Wer sich mit der Historie der NFL und ihrer Teams auskennt, weiß, dass in beiden Fällen de facto das gleiche Team nur unter anderem Namen auftritt. In der Tat ist es so, dass viele NFL-Teams bereits (mehrmals) ihre Namen geändert haben.  
Diese Tatsache stellt aus der Perspektive der Datenverarbeitung in der Tat ein Problem dar. Möchte man z. B. die Anzahl der verschiedenen Teams, die in der NFL gespielt haben, zählen, würde eine verfälschte Zahl entstehen, wenn man einfach nur die Anzahl der einzigartigen Team-Namen berechnet. Es ist daher nötig, den Teams einzigartige IDs zuzuweisen, um sie definitiv voneinander unterscheiden zu können. Dadurch ist egal, wie das Team zu einem gegebenen Zeitpunkt geheißen hat, da es trotzdem noch das gleiche Team (also die gleiche ID) ist. Dadurch entsteht ein ER-Modell, was im Folgenden aufgezeigt ist.  
Auch hierbei soll uns wieder eine Seite von [pro-football-reference.com](https://www.pro-football-reference.com/teams/), die alle Teams und ihre früheren Namen auflistet, helfen.  
  
<p align="center">
    <img src="Database-Viz.png" width="1000">
</p>

Zunächst wird die Seite mit den Team-Daten heruntergeladen

In [109]:
# Datei herunterladen

# URL, die gescraped werden soll
url = "https://www.pro-football-reference.com/teams/index.htm"

# Erstelle GET-Request der URL
response = requests.get(url)

# Prüfe, ob Request erfolgreich war
if response.status_code == 200:
    # Speichere Seite in einer HTML-Datei
    with open("data/teams.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Page saved as 'teams.html'")
else:
    print(f"Failed to download page. Status code: {response.status_code}")

Page saved as 'teams.html'


Nun muss die heruntergeladenen Seite `teams.html` geparsed werden, um alle Team-Namen zu finden. Dabei muss zwischen Team-Namen und Varianten unterschieden werden. Glücklicherweise sind diese im Quellcode durch unterschiedliche `css`-Klassen voneinander trennbar und können somit unterschieden werden.

In [94]:
# Datei nach Scores Parsen

with open("data/teams.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

all_names = dict()
all_teams = []

cntr = 0

for team in soup.select("table.stats_table#teams_active tbody tr:not([thead])"):
    name = team.find("th").text.strip("*")
    year_min = team.find("td", {"data-stat": "year_min"}).text.strip()
    year_max = team.find("td", {"data-stat": "year_max"}).text.strip()
    if not len(team.attrs.values()):
        cntr +=1
    all_teams.append([cntr,int(not len(team.attrs.values())),name,year_min,year_max])

    

comments = soup.find_all(string=lambda text: isinstance(text, Comment))
soup = BeautifulSoup(comments[22], "html.parser")

for team in soup.select("table.stats_table#teams_inactive tbody tr:not([thead])"):
    name = team.find("th").text.strip("*")
    year_min = team.find("td", {"data-stat": "year_min"}).text.strip()
    year_max = team.find("td", {"data-stat": "year_max"}).text.strip()
    if not len(team.attrs.values()):
        cntr +=1
    all_teams.append([cntr,int(not len(team.attrs.values())),name,year_min,year_max])

all_teams = pd.DataFrame(all_teams,columns=["team_id", "is_latest_name","team_name", "first_year", "last_year"])
all_teams.drop_duplicates(inplace=True, ignore_index=True)
all_teams.to_csv("data/teams.csv",index=False)

print("Done!")

Done!


Im nächsten Schritt sollen den Teams in `games.csv` nun tatsächlich ihre generierten IDs zugewiesen werden. Dabei werden auch noch weitere Kleinigkeiten gesäubert.

In [ ]:
games_df = pd.read_csv("data/games.csv")
teams_df = pd.read_csv("data/teams.csv")

games_new = []

games_df = games_df.drop("boxscore",axis=1)
games_df["game_location"] = games_df["game_location"].fillna(" ")

def find_team_id(source: pd.DataFrame, team: str, year: int):
    try:
        possible_teams = source[(source["team_name"] == team)]
        possible_teams = possible_teams[(year >= possible_teams["first_year"]) & (year <= possible_teams["last_year"] + 1)] # Die year-Spalten zeigen nur das erste Jahr der Saison.
        return int(possible_teams["team_id"].iloc[0])
    # Es gibt in der 1920 Saison Teams, die in der teams.csv nicht aufgelistet sind. Diese werden hier mit np.nan ersetzt, um die Endstände der Spiele trotzdem beizubehalten. 
    except IndexError:
        return np.nan

for index,game in tqdm(enumerate(games_df.values,1),desc=f"Applying Team-IDs to games"):
    game = (
        [index] +
		list(game[:4]) +
		[find_team_id(teams_df, game[4], int(game[2][:4]))] +
		list(game[5]) +
		[find_team_id(teams_df, game[6], int(game[2][:4]))] +
		list(game[7:]))
    games_new.append(game)

games_df = pd.DataFrame(
    games_new,
	columns=["game_id","week","day","date","outcome","winner_id","game_location","loser_id","pts_w","pts_l","yards_w","to_w","yards_l","to_l"]).convert_dtypes(convert_integer=True)

games_df["yards_w"] = games_df["yards_w"].replace(0,np.nan)
games_df["yards_l"] = games_df["yards_l"].replace(0,np.nan)
games_df.loc[games_df["date"] <= "1932-12-18", "to_w"] = np.nan
games_df.loc[games_df["date"] <= "1932-12-18", "to_l"] = np.nan

games_df.to_csv("data/games_cleaned.csv",index=False)

print(f"Done!")

Applying Team-IDs to games: 17949it [00:07, 2496.38it/s]

Done!


### Visualisierungen

In [ ]:
teams_df = pd.read_csv("data/teams.csv")

x = range(1920,2025)
y1 = [teams_df[(teams_df["is_latest_name"] == 1) & (year >= teams_df["first_year"]) & (year <= teams_df["last_year"])]["team_id"].count() for year in x]
y2 = [teams_df[(teams_df["is_latest_name"] == 1) & (year >= teams_df["first_year"]) & (2024 == teams_df["last_year"])]["team_id"].count() for year in x]

data = pd.DataFrame({"year": x,"#1": y1,"#2":y2})

fig = px.line(data,
              x="year",
              y="#1",
              range_y=[0,35],
              hover_name="year",
              hover_data={"#1":True,"#2": True,"year":False},
              color_discrete_sequence=["#013369"],
              labels={"year": "Year","#1": "# of Teams in the NFL","#2": "# of Teams currently in the NFL"},
              title="NFL Team-History")

fig.add_scatter(
              x=data["year"],
              y=data["#2"],
              hoverinfo="skip",
              name="# of Teams currently in the NFL",
              line={"dash": "dash", "color": "#d50a0a"},
              mode="lines")

fig.show()

In [155]:
teams_df = pd.read_csv("data/teams.csv")

teams_df["is_active"] = [year == 2024 for year in teams_df["last_year"]]
teams_df["is_active"] = teams_df["is_active"].astype(str)
teams_df["first_season"] = [f"{year}/{str(year + 1)[2:]}" for year in teams_df["first_year"]]
teams_df["last_season"] = [f"{year}/{str(year + 1)[2:]}" for year in teams_df["last_year"]]
teams_df["id_count"] = [sum(j == i for j in teams_df["team_id"]) for i in teams_df["team_id"]]
teams_df["first_year"] = [f"{year}-07-01" for year in teams_df["first_year"]]
teams_df["last_year"] = [f"{year + 1}-06-30" for year in teams_df["last_year"]]
teams_df["name_since"] = [teams_df[teams_df["team_id"] == i]["first_year"].max() for i in teams_df["team_id"]]
teams_df['team_id'] = teams_df['team_id'].astype(str)

order = list(pd.unique(teams_df[teams_df["is_latest_name"] == 1].sort_values(by=["last_year","first_year","name_since"])["team_id"]))

teams_df = teams_df[(teams_df["is_latest_name"] == 0) | (teams_df["id_count"] == 1)]

fig = px.timeline(
    teams_df,
	x_start="first_year",
	x_end="last_year",
	y="team_id",
	color="is_active",
	color_discrete_sequence=["#d50a0a","#013369"],
	category_orders={'team_id': order},
	range_x=["1920-01-01","2032-12-31"],
	custom_data=["team_name","team_id","first_season","last_season"],
	labels={
        "team_id": "Team",
		"is_active": "Still active?"},
	width=1500,
	height=900,
	title="History of Teams in the NFL: Timeline")

fig.update_traces(hovertemplate="<b>%{customdata[0]} </b><br><br>First Season: %{customdata[2]}<br>Last Season: %{customdata[3]}<extra><b>(%{customdata[1]})</b></extra>")
fig.update_yaxes(showticklabels=False)

fig.show()

### SCORIGAMI

In [ ]:
games_df = pd.read_csv("data/games_cleaned.csv")

score_counts = games_df.groupby(["pts_w", "pts_l"]).size().reset_index(name="count")

custom_colorscale = [
    [0, "white"],
    [0.00001, "darkolivegreen"],
    [0.03, "green"],
    [1/3, "yellow"],
    [1, "red"]]

fig = px.density_heatmap(
    score_counts,
    x="pts_w",
    y="pts_l",
    z="count",
    color_continuous_scale=custom_colorscale,
    labels={
        "pts_w": "Winning Points",
		"pts_l": "Losing Points",
		"count": "Anzahl Spiele"},
    title="Scorigami-Heatmap: Frequency of Final Scores",
    nbinsx=100,
    nbinsy=100,
    width=1500,
    height=1000)

path = "M 6.5 0.5 H 7.5 V 1.5 H 6.5 V 0.5 M -0.5 0.5 H 0.5 V -0.5 H 1.5 V 0.5 H 5.5 V 1.5 H 0.5" + "".join([f"H {0.5 + i} V {1.5 + i} " for i in range(51)]) + "H -0.5 Z"
fig.add_shape(
    type="path",
    path=path,
    xref="x",
    yref="y",
    line_width=0,
    fillcolor="black")

fig.add_shape(
    type="rect",
    x0=-0.5,
    x1=73.5,
    y0=-0.5,
    y1=51.5,
    line_width=2,
    line_color="black")

fig.update_layout(
    margin=dict(t=90, r=10, b=10, l=60),
    hoverlabel_bgcolor="red",
    paper_bgcolor="azure",
    title=dict(
        font_weight=1000,
        x=0.015,
        y=0.98),
    coloraxis_colorbar=dict(
        title="",
        bgcolor="white",
        bordercolor="black",
        borderwidth=2,
        thickness=20),
    xaxis=dict(
        showspikes=True,
        spikethickness=0,
        side="top",
        dtick=2,
        ticks="outside",
        ticklen=3,
        tickcolor="dimgray",
        title_standoff=5,
        title_font_variant="small-caps"),
    yaxis=dict(
        showspikes=True,
        spikethickness=0,
        autorange="reversed",
        dtick=2,
        ticks="outside",
        ticklen=3,
        tickcolor="dimgray",
        title_standoff=5,
        title_font_variant="small-caps"))

fig.update_traces(hovertemplate="Score<br># Games <extra><b>%{x} - %{y}<br>%{z}</b></extra>")

fig.show()

# Grafik-Erweiterung: Colorscale nach Zeit - Wann ist dieser Score das erste (letzte?) mal passiert?

In [ ]:
games_df = pd.read_csv("data/games_cleaned.csv")

score_counts = games_df.groupby(["pts_w", "pts_l"]).size().reset_index(name="count")

custom_colorscale = [
    [0, "white"],
    [0.00001, "darkolivegreen"],
    [0.03, "green"],
    [0.33, "yellow"],
    [1, "red"]]

fig = go.Figure(data=go.Heatmap(
    x=score_counts["pts_w"],
    y=score_counts["pts_l"],
    z=[i or 0 for i in score_counts["count"]],
    zmin=-1,
    zmax=300,
    colorscale=custom_colorscale
    # labels={
    #     "pts_w": "Winning Points",
	# 	"pts_l": "Losing Points",
	# 	"count": "Anzahl Spiele"},
    # title="Scorigami-Heatmap: Frequency of Final Scores",
    # nbinsx=100,
    # nbinsy=100,
    ))

path = "M 6.5 0.5 H 7.5 V 1.5 H 6.5 V 0.5 M -0.5 0.5 H 0.5 V -0.5 H 1.5 V 0.5 H 5.5 V 1.5 H 0.5" + "".join([f"H {0.5 + i} V {1.5 + i} " for i in range(51)]) + "H -0.5 Z"
fig.add_shape(
    type="path",
    path=path,
    xref="x",
    yref="y",
    line_width=0,
    fillcolor="black")

fig.update_traces(customdata=["count"],hovertemplate="Score<br># Games <extra><b>%{x} - %{y}<br>%{z}</b></extra>")
fig.update_layout(
    width=1500,
    height=1050,
    # coloraxis_colorbar_title="",
    # colorscale_sequential=custom_colorscale,
    xaxis=dict(
        range=[-0.5,73.5],
        side="top",
        dtick=1,
        ticks="outside",
        ticklen=3,
        tickcolor="black"),
    yaxis=dict(
        range=[51,-0.5],
        # autorange="reversed",
        dtick=1,
        ticks="outside",
        ticklen=3,
        tickcolor="black"))

fig.show()

AttributeError: 'DataFrame' object has no attribute 'append'

In [178]:
def coin_change(target, coins):
    wallets = [[coin] for coin in coins]
    new_wallets = []
    sol = []
    
    if target in coins:
        sol.append([target])

    while wallets:
        for wallet in wallets:
            s = sum(wallet)
            for coin in coins:
                if coin >= wallet[-1]:
                    if s + coin < target:
                        new_wallets.append(wallet + [coin])
                    elif s + coin == target:
                        sol.append(wallet + [coin])
        wallets = new_wallets
        new_wallets = []
    return sol

def calc_prob(perms,percs):
	cum_prob = 0
	for i in perms:
		prob = 1
		for j in i:
			prob *= percs[j]
		cum_prob += prob
	return cum_prob

percentages = {2: .01, 3: .26, 6: .03, 7: .67, 8: .03}
vals = percentages.keys()

x = range(2,74)
y = [calc_prob(coin_change(i,vals),percentages)*i for i in x]

data = pd.DataFrame({"pts": x,"%": y})
fig = px.bar(data,
              x="pts",
              y="%")

fig.show()